### IMPORTS

In [ ]:
import whisper
import speech_recognition as sr
import torch
import numpy as np
import wave
import ollama
import json
import msvcrt
import pybullet as p
import pybullet_data
import time
import math 
import threading

### LLM MODEL

In [2]:
LLM_MODEL = "mistral"

### VOICE RECOGNITION

In [ ]:
MODEL_NAME = "turbo" 

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading {MODEL_NAME} model on {device}...")
model = whisper.load_model(MODEL_NAME, device=device)

def audio_by_user():
    recognizer = sr.Recognizer()
    with sr.Microphone(sample_rate=16000) as source: # Force 16kHz
        print("Say: 'Robot, move forward one meter.'")
        
        print("Listening...")
        audio = recognizer.listen(source, timeout=6, phrase_time_limit=10)
        
        with open("debug_audio.wav", "wb") as f:
            f.write(audio.get_wav_data())
        print("Audio saved to 'debug_audio.wav'. Listen to it to check for noise/static.")

        # Transcribe
        audio_data = np.frombuffer(audio.get_raw_data(), np.int16).flatten().astype(np.float32) / 32768.0
        result = model.transcribe(audio_data, language="en", fp16=torch.cuda.is_available())
        
        user_speech = print(f"\nWhisper Result: \"{result['text'].strip()}\"")

    return user_speech


Loading turbo model on cuda...


### LLM PROCESSING FOR ROBOT CODE GENERATION

In [4]:
def get_mission_plan(user_input):
    print(f"[BRAIN] Processing logic with {LLM_MODEL}...")

    # STRICT System Prompt
    # We tell the LLM exactly what objects exist so it doesn't hallucinate "yellow_box"
    system_manual = """
    You are a Robot Logic Unit.
    
    ENVIRONMENT:
    - OBJECTS: ['red_box', 'green_box', 'blue_box']
    - LOCATIONS: ['home_position', 'table_center']
    
    COMMANDS ALLOWED:
    - 'pick_up(target)'
    - 'move_to(target)'
    - 'place_at(target)'
    
    TASK:
    Output a JSON object containing a list of actions based on the user's command.
    
    Example Input: "Grab the red box and put it on the table."
    Example Output: {
        "plan": [
            {"action": "pick_up", "target": "red_box"},
            {"action": "place_at", "target": "table_center"}
        ]
    }
    """

    try:
        response = ollama.generate(
            model=LLM_MODEL,
            system=system_manual,
            prompt=f"User Command: {user_input}",
            format="json",  # Forces JSON output mode in Ollama
            options={"temperature": 0} # 0 makes it deterministic (logic > creativity)
        )
        
        raw_response = response['response']
        print(f"[BRAIN] Raw Model Output: {raw_response}") # Debug print
        
        # Parse JSON
        data = json.loads(raw_response)
        
        # Normalize: Try to find a list of actions
        if "plan" in data:
            return data["plan"]
        elif isinstance(data, list):
            return data
        else:
            # If it returns a single object, wrap it in a list
            return [data]

    except Exception as e:
        print(f"[BRAIN] Error: {e}")
        return []

### INITIALIZE SIMULATION IN PYBULLET

In [5]:
ROBOT_ID = None
END_EFFECTOR_INDEX = 6
OBJECTS = {}   # Stores ID of red_box, blue_box, etc.
LOCATIONS = {} # Stores coordinates of home, drop_zone
MAGNET_CONSTRAINT = None # Stores the ID of the connection when holding an object
SIM_ACTIVE = True

def setup_world():
    global ROBOT_ID, OBJECTS, LOCATIONS
    
    # Initialize
    p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.81)

    # Load Floor
    p.loadURDF("plane.urdf")
    
    # Load Table (Rotated 90 deg, set at proper height)
    p.loadURDF("table/table.urdf", basePosition=[0.5, 0, -0.65], baseOrientation=p.getQuaternionFromEuler([0, 0, 1.57]))

    # Load Robot (Kuka IIWA)
    # Mounted at the edge (0.1, 0, 0) sitting ON the table
    ROBOT_ID = p.loadURDF("kuka_iiwa/model.urdf", basePosition=[0.1, 0, 0], useFixedBase=True)
    
    # Define Locations
    LOCATIONS = {
        "home": [0.3, 0, 0.6],
        "drop_zone": [0.0, 0.5, 0.4] 
    }

    # Spawn Boxes (Helper function called inside)
    OBJECTS["red_box"] = spawn_box([0.6, 0.2, 0.1], [1, 0, 0, 1])
    OBJECTS["green_box"] = spawn_box([0.6, 0.0, 0.1], [0, 1, 0, 1])
    OBJECTS["blue_box"] = spawn_box([0.6, -0.2, 0.1], [0, 0, 1, 1])

def spawn_box(pos, color_rgba):
    # Spawns a box and colors it
    box_id = p.loadURDF("cube.urdf", basePosition=pos, globalScaling=0.08)
    p.changeVisualShape(box_id, -1, rgbaColor=color_rgba)
    return box_id

def get_coordinate(name):
    # Returns [x,y,z] for a name (box or location)
    if name in OBJECTS:
        pos, _ = p.getBasePositionAndOrientation(OBJECTS[name])
        return list(pos)
    elif name in LOCATIONS:
        return LOCATIONS[name]
    return None

def move_arm(target_pos):
    print(f"[ROBOT] Moving to {target_pos}...")
    
    # Orientation: Gripper pointing down
    orn = p.getQuaternionFromEuler([0, 3.14, 0])
    
    # Inverse Kinematics calculation
    joint_poses = p.calculateInverseKinematics(
        ROBOT_ID, 
        END_EFFECTOR_INDEX, 
        target_pos, 
        orn,
        maxNumIterations=100
    )

    # Apply movement to motors
    for i in range(len(joint_poses)):
        p.setJointMotorControl2(ROBOT_ID, i, p.POSITION_CONTROL, joint_poses[i], force=500)
    
    # Small blocking delay to let it travel
    time.sleep(1.5)

def toggle_magnet(state):
    global MAGNET_CONSTRAINT
    
    # Get current position of the gripper
    ee_pos = p.getLinkState(ROBOT_ID, END_EFFECTOR_INDEX)[0]
    
    if state == "on":
        # Find closest object
        closest_obj = None
        min_dist = 0.2 # 20cm search radius

        for name, obj_id in OBJECTS.items():
            obj_pos, _ = p.getBasePositionAndOrientation(obj_id)
            dist = np.linalg.norm(np.array(ee_pos) - np.array(obj_pos))
            if dist < min_dist:
                min_dist = dist
                closest_obj = obj_id
        
        # If found and not already holding something
        if closest_obj is not None and MAGNET_CONSTRAINT is None:
            print(f"[ROBOT] Magnet ON. Attached to object {closest_obj}")
            # Create a fixed link between gripper and box
            MAGNET_CONSTRAINT = p.createConstraint(
                ROBOT_ID, END_EFFECTOR_INDEX, 
                closest_obj, -1, 
                p.JOINT_FIXED, [0, 0, 0], [0, 0, 0], [0, 0, 0]
            )
        else:
            print("[ROBOT] Magnet ON. No object found nearby.")

    elif state == "off":
        if MAGNET_CONSTRAINT is not None:
            print("[ROBOT] Magnet OFF. Object dropped.")
            p.removeConstraint(MAGNET_CONSTRAINT)
            MAGNET_CONSTRAINT = None

def physics_thread():
    # This runs in the background to keep gravity working
    while SIM_ACTIVE:
        p.stepSimulation()
        time.sleep(1./240.)